In [1]:
from threading import Thread
import concurrent.futures
import time

import sqlite3

import pandas as pd
import numpy as np
from urllib.request import urlopen  
import json
import requests

getkey = ! cat ../../apikey.txt
apikey = getkey[0]

from fhelp import get_appended,processDatesIT
from FMPGet import FMPGet
from getFunctions import ReqInfo

In [2]:
a = FMPGet(apikey,tickerset='sp500')

In [3]:
%%time 
a.run(verbose=True)

G1_CAL_EARNINGSCALENDAR_UPC SUCCESS
G1_MI_MAJORINDEXES SUCCESS
G1_MI_SP500HIST SUCCESS
G1_MI_SP500LIST SUCCESS
G1_CV_DELISTED SUCCESS
G1_INST_CIKLIST SUCCESS
G1_CV_ETFS SUCCESS
G1_MI_DOWJLIST SUCCESS
G1_MI_NASDAQLIST SUCCESS
G1_MI_DOWJHISTLIST SUCCESS
G1_MI_NASDAQHISTLIST SUCCESS
G1_COM_SYMBOLS SUCCESS
G1_COM_PRICES SUCCESS
G1_ETF_SYMBOLS SUCCESS
G1_ETF_PRICES SUCCESS
G1_MF_SYMBOLS SUCCESS
G1_MF_PRICES SUCCESS
G1_CV_SYMBOLS SUCCESS
G1_CV_TRADABLE SUCCESS
Retrieving sp500-specific tickerset(funds,ciks,indexes,etc)
Retrieving stocks and indices
Retrieving etfs and commods
CV_PROFILE SUCCESS
Retrieving funds, inst, and ciks
INST_MUTFUNDHOLDERS SUCCESS
INST_HOLDERS SUCCESS
Retrieving ticker-dependent data
CV_PROFILE SUCCESS
CV_QUOTE SUCCESS
CV_EXECUTIVES SUCCESS
CV_FS_INCOME_YR SUCCESS
CV_FS_INCOME_QTR SUCCESS
CV_FS_BALANCESHEET_YR SUCCESS
CV_FS_BALANCESHEET_QTR SUCCESS
CV_FS_CASHFLOW_YR SUCCESS
CV_FS_CASHFLOW_QTR SUCCESS
CV_FS_INCOMEGROWTH_YR SUCCESS
CV_FS_BALANCEGROWTH_YR SUCCESS
CV_FS_C

Exception ignored in: <generator object as_completed at 0x7fb8020ef350>
Traceback (most recent call last):
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 250, in as_completed
    ref_collect=(fs, pending))
KeyboardInterrupt: 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/verbeck/Desktop/Finance/Technology/Project/Setup/getFunctions.py", line 213, in getTableMT
    tempdf = tempdf.append(tab)
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py", line 7983, in append
    sort=sort,
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/concat.py", line 295, in concat
    sort=sort,
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/concat.py", line 373, in __init__
    obj._consolidate_inplace()
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 5538, in _consolidate_inplace
    self._protect_consolidate(f)
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 5526, in _protect_consolidate
    result = f()
  File "/Users/verbeck/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 5536, in f
    self._mgr =

TypeError: object of type 'NoneType' has no len()

In [10]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [11]:
independents = urls[[name for name in urls.index if name[0] == 'G']]
dependents = urls[[name for name in urls.index if name[0] != 'G' and name[0] != 'U']]
unused = urls[[name for name in urls.index if name[0] == 'U']]

In [12]:
pd.set_option('max_rows',None)
dependents

P1b1000_CV_PROFILE              https://financialmodelingprep.com/api/v3/profi...
H1b1000_CV_QUOTE                https://financialmodelingprep.com/api/v3/quote...
S1_CV_EXECUTIVES                https://financialmodelingprep.com/api/v3/key-e...
S1_CV_FS_INCOME_YR              https://financialmodelingprep.com/api/v3/incom...
S1_CV_FS_INCOME_QTR             https://financialmodelingprep.com/api/v3/incom...
S1_CV_FS_BALANCESHEET_YR        https://financialmodelingprep.com/api/v3/balan...
S1_CV_FS_BALANCESHEET_QTR       https://financialmodelingprep.com/api/v3/balan...
S1_CV_FS_CASHFLOW_YR            https://financialmodelingprep.com/api/v3/cash-...
S1_CV_FS_CASHFLOW_QTR           https://financialmodelingprep.com/api/v3/cash-...
S1_CV_FS_INCOMEGROWTH_YR        https://financialmodelingprep.com/api/v3/incom...
S1_CV_FS_BALANCEGROWTH_YR       https://financialmodelingprep.com/api/v3/balan...
S1_CV_FS_CASHGROWTH_YR          https://financialmodelingprep.com/api/v3/cash-...
S1_CV_RATIOS_TTM

In [ ]:
! rm ../../FinancesTEST.db

In [ ]:
urls = pd.read_json('Resources/urls.json',typ='series')

In [ ]:
#conn = sqlite3.connect('../../Finances.db')
conn = sqlite3.connect('FinancesTEST.db')
c = conn.cursor()

In [ ]:
def load_url(pair,apikey):
    name = pair[0]
    url = pair[1]
    tab = get_appended(url,apikey,name)
    return tab

In [ ]:
%%time
rate_limit = 10
offset = 1/rate_limit
max_workers = rate_limit -1
with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url,pair,apikey): pair for pair in independents.iteritems()}
    for future in concurrent.futures.as_completed(future_to_url):
        time.sleep(offset)
        pair = future_to_url[future]
        try:
            tab = future.result()
        except Exception as exc:
            print(pair,'FAILED',exc)
            
        else:
            c.execute('drop table if exists '+ pair[0])
            tab.to_sql(pair[0],con=conn,index=False,if_exists='append')
            print(pair, 'SUCCESS')

In [8]:
|

SyntaxError: invalid syntax (<ipython-input-8-4b37ef281455>, line 1)

In [14]:
#setup database
###FULL lists for full set processing
stocks = list(pd.io.sql.read_sql('select symbol from G1_CV_SYMBOLS;',con=conn).symbol)
etfs  = list(pd.io.sql.read_sql('select symbol from G1_ETF_SYMBOLS;',con=conn).symbol)
etfs2  = list(pd.io.sql.read_sql('select symbol from G1_CV_ETFS;',con=conn).symbol)
indexes = list(pd.io.sql.read_sql('select symbol from G1_MI_MAJORINDEXES;',con=conn).symbol)
ciks = ['000'+str(c) for c in list(pd.io.sql.read_sql('select cik from G1_INST_CIKLIST',con=conn).cik)]
funds = list(pd.io.sql.read_sql('select symbol from G1_MF_SYMBOLS;',con=conn).symbol)
commods = list(pd.io.sql.read_sql('select symbol from G1_COM_SYMBOLS;',con=conn).symbol)
##Light cleanup, nothing perfect –– could set this up using sql. 

#merge etf lists
etfs_c = list(set(etfs).union(set(etfs2)))
#leave stocks only 
stocks_c = list(set(stocks) - set(etfs_c + indexes + ciks + funds + commods))

In [15]:
#setup subset of tickers
dow_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_DOWJLIST;',con=conn).symbol)
nasdaq_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_NASDAQLIST;',con=conn).symbol)
sp500_tickers = list(pd.io.sql.read_sql('select symbol from G1_MI_SP500LIST;',con=conn).symbol)

In [13]:
#define subset to work with
stocksub = dow_tickers
#stocksub = sp500_tickers
indexsub = ['^IXIC','^GSPC','^DJI']
#indexsub = ['^IXIC','^GSPC','^DJI','^VIX','^RUA','^FTSE','^NDX','^N225','GDAXI']

In [14]:
#get top US etfs proportionate to stock list
num_etfs = int(np.ceil(len(stocksub)/10))
profile = get_appended(dependents['P1b1000_CV_PROFILE'],apikey,'P1b1000_CV_PROFILE',tickers = etfs_c)
etfsub = list(profile[profile.country == 'US'].sort_values(by='volAvg',ascending=False).symbol[:num_etfs+1])
tickersub = stocksub + etfsub

In [15]:
#get mutual fund data
fundtable = get_appended(dependents['T1_INST_MUTFUNDHOLDERS'],apikey,'T1_INST_MUTFUNDHOLDERS',tickers=tickersub,verbose=False)
fundsub = list(fundtable.holder.value_counts().index)

In [16]:
mfs = pd.io.sql.read_sql('select * from G1_MF_SYMBOLS;',con=conn)
fundsub = list(np.random.choice(list(mfs[mfs.name.isin(fundsub)].symbol),int(np.ceil(len(stocksub)/3))))

In [17]:
#establish subset of commods
#commodsub = commods[:3]
commodsub = commods

In [18]:
#get ciks of institutional holders for ciksub
instable = get_appended(dependents['T1_INST_HOLDERS'],apikey,'T1_INST_HOLDERS',tickers=tickersub)
instsub = list(instable.holder.value_counts().index)
instsub = list(np.random.choice(instsub,500)) #account for many missing inst-cik mappings

In [19]:
#retrieve ciks
ciklist = pd.io.sql.read_sql('select * from G1_INST_CIKLIST',con=conn)
ciksub = list(ciklist[ciklist.name.apply(lambda x: x.lower()).isin([i.lower() for i in instsub])].cik)
ciksub = ['000'+str(c) for c in (ciksub)]
ciksub = [c for c in ciksub if len(c)==10]

In [20]:
#dictionary – nothing commodity, crypto, etc. specific
d = {'H':stocksub + etfsub + commodsub + fundsub,
     'P':stocksub + etfsub + fundsub,
     'T':stocksub + etfsub,
     'S':stocksub,
     'E':etfsub,
     'C':commodsub,
     'F':ciksub,
     'I':indexsub,
     'M':fundsub}

In [33]:
tempdf = pd.DataFrame()
apikey = apikey
name = "S2_CV_DCF_YR"
url = "https://financialmodelingprep.com/api/v3/historical-discounted-cash-flow/ ?limit=50&apikey="
tickers = dow_tickers
batch = False
reqInfo = ReqInfo(10,1,5)
verbose = True
MAX_WORKERS = 5
OFFSET = 1
addsymbol = False

In [34]:
def getRequestJ2(apikey,url,ticker,offset,verbose=False):
    '''Return partial dataframe for ticker'''
    
    time.sleep(offset)
    df = None
    #create valid URL
    final_url = url.replace(' ',ticker) + apikey
    try:
        jsonx = pd.read_json(final_url)
        symbname = jsonx.columns[0]
    
        df = pd.DataFrame(jsonx.iloc[0,1])
        df[symbname] = jsonx.loc[0,symbname]
    except Exception as e :
        print(e, '...ticker {} failed'.format(ticker))
        
    return df

In [35]:
%%time
with concurrent.futures.ThreadPoolExecutor(max_workers=9) as executor:
    
    #create future for each batch
    future_to_tickers = {executor.submit(getRequestJ2,apikey,url,ticker,OFFSET,verbose=True): ticker for ticker in tickers}
    
    for future in concurrent.futures.as_completed(future_to_tickers,timeout=None):
        
        ticker = future_to_tickers[future]
        try:
            tab = future.result()
        except Exception as e:
            print(ticker,'FAILED',e)
            
        else:
            if addsymbol:
                tempdf['symbol'] = ticker
            tempdf = tempdf.append(tab)
            print(ticker, 'SUCCESS')

CRM SUCCESS
V SUCCESS
NKE SUCCESS
WBA SUCCESS
WMT SUCCESS
VZ SUCCESS
INTC SUCCESS
UNH SUCCESS
TRV SUCCESS
JNJ SUCCESS
MCD SUCCESS
DIS SUCCESS
JPM SUCCESS
AMGN SUCCESS
BA SUCCESS
DOW SUCCESS
AAPL SUCCESS
CAT SUCCESS
CSCO SUCCESS
GS SUCCESS
MSFT SUCCESS
HD SUCCESS
PG SUCCESS
MRK SUCCESS
IBM SUCCESS
KO SUCCESS
HON SUCCESS
CVX SUCCESS
AXP SUCCESS
MMM SUCCESS
CPU times: user 527 ms, sys: 33 ms, total: 560 ms
Wall time: 5.33 s


In [39]:
len(tempdf.symbol.value_counts())

30